In [24]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [25]:
model = DiscreteBayesianNetwork(
    [
        ("Intelligence","Grade"),
        ("StudyHour","Grade"),
        ("Difficulty","Grade"),
        ("Grade", "Pass")
    ]
)

In [26]:
#independent Probablities
cpd_int = TabularCPD("Intelligence",2,[[0.7],[0.3]], state_names={"Intelligence" : ["High","Low"]})
cpd_stu = TabularCPD("StudyHour",2,[[0.6],[0.4]], state_names={"StudyHour" : ["Sufficient","Insufficient"]})
cpd_dif = TabularCPD("Difficulty",2,[[0.4],[0.6]], state_names={"Difficulty" : ["Hard","Easy"]})


In [27]:
#dependent Probablities

cpd_grd = TabularCPD(
    "Grade",3,
    [
        [0.7, 0.5, 0.5, 0.3, 0.4, 0.2, 0.2, 0.1],  # A
        [0.2, 0.3, 0.3, 0.4, 0.4, 0.4, 0.3, 0.3],  # B
        [0.1, 0.2, 0.2, 0.3, 0.2, 0.4, 0.5, 0.6]   # C
    ],
    evidence=["Intelligence","StudyHour","Difficulty"],
    evidence_card=[2,2,2],
    state_names={
        'Grade': ['A', 'B', 'C'],
        'Intelligence': ['High', 'Low'],
        'StudyHour': ['Sufficient', 'Insufficient'],
        'Difficulty': ['Hard', 'Easy']
    }
    
)
cpd_pass = TabularCPD(
    "Pass",2
    ,[[0.95,0.80,0.50],[0.05,0.20,0.50]],
    evidence=["Grade"]
    ,evidence_card=[3]
    , state_names={
        "Pass": ["Yes","No"],
        'Grade': ['A', 'B', 'C']
    }
)

In [28]:
model.add_cpds(cpd_int,cpd_stu,cpd_dif,cpd_grd,cpd_pass)
model.check_model()

True

In [29]:
inference = VariableElimination(model)

In [30]:
# Q1: P(Pass=Yes | StudyHours=Sufficient, Difficulty=Hard)
q1 = inference.query(variables=['Pass'], evidence={'StudyHour': 'Sufficient', 'Difficulty': 'Hard'})
print("\nP(Pass=Yes | StudyHour=Sufficient, Difficulty=Hard):\n", q1)

# Q2: P(Intelligence=High | Pass=Yes)
q2 = inference.query(variables=['Intelligence'], evidence={'Pass': 'Yes'})
print("\nP(Intelligence=High | Pass=Yes):\n", q2)



P(Pass=Yes | StudyHour=Sufficient, Difficulty=Hard):
 +-----------+-------------+
| Pass      |   phi(Pass) |
+===========+=============+
| Pass(Yes) |      0.8525 |
+-----------+-------------+
| Pass(No)  |      0.1475 |
+-----------+-------------+

P(Intelligence=High | Pass=Yes):
 +--------------------+---------------------+
| Intelligence       |   phi(Intelligence) |
+====================+=====================+
| Intelligence(High) |              0.7285 |
+--------------------+---------------------+
| Intelligence(Low)  |              0.2715 |
+--------------------+---------------------+
